In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://ppa

In [2]:
pip install quinn   

In [201]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType,IntegerType,DateType,DoubleType
from pyspark.sql import SparkSession #DataFrame objeleri yaratmak için gerekli
from pyspark.sql.types import StructType,StructField,ArrayType
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import quinn
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark import SparkContext

In [191]:
spark = SparkSession.builder\
            .appName("Spark Dataframe Intro")\
            .getOrCreate()
            
leaf = spark.read\
          .option("inferSchema", "True")\
          .option("header", "true")\
          .csv("leaf.csv")
leaf.printSchema()
leaf.createOrReplaceTempView("leaf")
leaf = spark.sql("select * from leaf")
leaf.take(2)                   

root
 |-- 10: integer (nullable = true)
 |-- 11: integer (nullable = true)
 |-- 0.72694: double (nullable = true)
 |-- 1.4742: double (nullable = true)
 |-- 0.32396: double (nullable = true)
 |-- 0.98535: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 0.83592: double (nullable = true)
 |-- 0.0046566: double (nullable = true)
 |-- 0.0039465: double (nullable = true)
 |-- 0.04779: double (nullable = true)
 |-- 0.12795: double (nullable = true)
 |-- 0.016108: double (nullable = true)
 |-- 0.0052323: double (nullable = true)
 |-- 0.00027477: double (nullable = true)
 |-- 1.1756: double (nullable = true)



[Row(10=1, 11=2, 0.72694=0.74173, 1.4742=1.5257, 0.32396=0.36116, 0.98535=0.98152, 16=0.99825, 0.83592=0.79867, 0.0046566=0.0052423, 0.0039465=0.0050016, 0.04779=0.02416, 0.12795=0.090476, 0.016108=0.0081195, 0.0052323=0.002708, 0.00027477=7.4846e-05, 1.1756=0.69659),
 Row(10=1, 11=3, 0.72694=0.76722, 1.4742=1.5725, 0.32396=0.38998, 0.98535=0.97755, 16=1.0, 0.83592=0.80812, 0.0046566=0.0074573, 0.0039465=0.010121, 0.04779=0.011897, 0.12795=0.057445, 0.016108=0.0032891, 0.0052323=0.00092068, 0.00027477=3.7886e-05, 1.1756=0.44348)]

For renaming the column names the function is created below

In [193]:
def rename_col(s):
 mapping = {
    '10' : 'label','11' : 'Specimen Number', '0.72694' :'Eccentricity','1.4742' :'Aspect Ratio','0.32396' : 'Elongation',
    '0.98535' : 'Solidity', '16' : 'Stochastic Convexity','0.83592' : 'Isoperimetric Factor', '0.0046566' : 'Maximal Indentation Depth',
    '0.0039465' : 'Lobedness', '0.04779' : 'Average Intensity', '0.12795' : 'Average Contrast', '0.016108' : 'Smoothness',
    '0.0052323' : 'Third moment',  '0.00027477' : 'Uniformity',  '1.1756' : 'Entropy'
      }
 return mapping[s]
leaf = leaf.transform(quinn.with_columns_renamed(rename_col))
 

ın order to check null values the function is created below

In [194]:
def null_checker(leaf):
  leaf.select([count(when(isnan(c), c)).alias(c) for c in leaf.columns]).show()
  return leaf
null_checker(leaf)

+-----+---------------+------------+------------+----------+--------+--------------------+--------------------+-------------------------+---------+-----------------+----------------+----------+------------+----------+-------+
|label|Specimen Number|Eccentricity|Aspect Ratio|Elongation|Solidity|Stochastic Convexity|Isoperimetric Factor|Maximal Indentation Depth|Lobedness|Average Intensity|Average Contrast|Smoothness|Third moment|Uniformity|Entropy|
+-----+---------------+------------+------------+----------+--------+--------------------+--------------------+-------------------------+---------+-----------------+----------------+----------+------------+----------+-------+
|    0|              0|           0|           0|         0|       0|                   0|                   0|                        0|        0|                0|               0|         0|           0|         0|      0|
+-----+---------------+------------+------------+----------+--------+--------------------+------

DataFrame[label: int, Specimen Number: int, Eccentricity: double, Aspect Ratio: double, Elongation: double, Solidity: double, Stochastic Convexity: double, Isoperimetric Factor: double, Maximal Indentation Depth: double, Lobedness: double, Average Intensity: double, Average Contrast: double, Smoothness: double, Third moment: double, Uniformity: double, Entropy: double]

for transforming columns the function is created below

In [195]:
def transformers(leaf):
  vec = VectorAssembler(inputCols=leaf.columns[2:],outputCol="features")
  leaf = vec.transform(leaf)
  return leaf
transformers(leaf)
leaf.show(1)
leaf =   transformers(leaf)


+-----+---------------+------------+------------+----------+--------+--------------------+--------------------+-------------------------+---------+-----------------+----------------+----------+------------+----------+-------+
|label|Specimen Number|Eccentricity|Aspect Ratio|Elongation|Solidity|Stochastic Convexity|Isoperimetric Factor|Maximal Indentation Depth|Lobedness|Average Intensity|Average Contrast|Smoothness|Third moment|Uniformity|Entropy|
+-----+---------------+------------+------------+----------+--------+--------------------+--------------------+-------------------------+---------+-----------------+----------------+----------+------------+----------+-------+
|    1|              2|     0.74173|      1.5257|   0.36116| 0.98152|             0.99825|             0.79867|                0.0052423|0.0050016|          0.02416|        0.090476| 0.0081195|    0.002708| 7.4846E-5|0.69659|
+-----+---------------+------------+------------+----------+--------+--------------------+------

One-vs-Rest classifier (a.k.a. One-vs-All)

In [254]:
def one_vs_Rest_classifier(leaf):
  global method
  global maxIter
  global tol
  global resultseto_vs_rest
  method = "One-vs-Rest classifier"
  maxIter=10
  tol=13-6
  (train, test) = leaf.randomSplit([0.8, 0.2])

  # instantiate the base classifier.
  lr = LogisticRegression(maxIter=maxIter, tol=tol, fitIntercept=True)

  # instantiate the One Vs Rest Classifier.
  ovr = OneVsRest(classifier=lr)

  # train the multiclass model.
  ovrModel = ovr.fit(train)

  # score the model on test data.
  predictions = ovrModel.transform(test)

  # obtain evaluator.
  evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

  # compute the classification error on test data.
  accuracy = evaluator.evaluate(predictions)
  resultseto_vs_rest = ([{ 
    "Method" : method,
    "Parameters" :  ("Param1=" + str(maxIter),
                     "Param2=" + str(tol)),
    "Accuracy" :  accuracy                    
                    }])
  return resultseto_vs_rest
one_vs_Rest_classifier(leaf)  
deptColumns = ["Accuracy","Method","Parameters"]
resultseto_vs_rest = spark.createDataFrame(data=resultseto_vs_rest, schema = deptColumns)
resultseto_vs_rest.createOrReplaceTempView("one_vs_rest_result")
result_of_onevs = spark.sql("select Method, Parameters, Accuracy from one_vs_rest_result").createOrReplaceTempView("one_vs_rest_result")

Decision Tree classifier

In [267]:
def decisiontreeclassifier(leaf):
  global resultsetdt
  method = "Decision Tree Classifier"
  trainsize = 0.7
  testsize = 0.3


  # Split the data into training and test sets (30% held out for testing)
  (trainingData, testData) = leaf.randomSplit([trainsize, testsize])

  # Train a DecisionTree model.
  dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

  # Chain indexers and tree in a Pipeline
  pipeline = Pipeline(stages=[dt])

  # Train model.  This also runs the indexers.
  model = pipeline.fit(trainingData)

  # Make predictions.
  predictions = model.transform(testData)

  # Select example rows to display.
  predictions.select("prediction", "label", "features")

  # Select (prediction, true label) and compute test error
  evaluator = MulticlassClassificationEvaluator(
      labelCol="label", predictionCol="prediction", metricName="accuracy")
  accuracy = evaluator.evaluate(predictions)
  resultsetdt = [{ 
    "Method" :  method,
    "Parameters" :  ("Param1=" + str(trainsize),
                     "Param2=" + str(testsize)),
    "Accuracy"  :  accuracy
                    }]
  return resultsetdt
decisiontreeclassifier(leaf)
resultsetdt
deptColumns = ["Accuracy","Method","Parameters"]
resultset = spark.createDataFrame(data=resultsetdt, schema = deptColumns)
resultset.createOrReplaceTempView("decisiontreeresults")
result_of_onedt = spark.sql("select Method, Parameters, Accuracy from decisiontreeresults").createOrReplaceTempView("decisiontreeresults")

random forest classifier

In [277]:

methodrf = "Random Forest Classifier"
rfClassifier = RandomForestClassifier()
trainDF, testDF = leaf.randomSplit([0.7,0.3],seed=123)
eva = MulticlassClassificationEvaluator(metricName="accuracy")
params = ParamGridBuilder()\
    .addGrid(rfClassifier.maxDepth, [6,7,8,9,10,11]) \
    .addGrid(rfClassifier.numTrees, [6,7,8,9,10,11])\
    .addGrid(rfClassifier.impurity, ['gini',"entropy"])\
    .build()

validator = TrainValidationSplit(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                trainRatio=0.8
                                )


model = validator.fit(trainDF)

#print("Num Trees : ",model.bestModel.getNumTrees)
#print("Max Depth : ",model.bestModel._java_obj.getMaxDepth())
#print("Impurtiy : ",model.bestModel._java_obj.getImpurity())

sonucDF = model.transform(testDF)

basari = eva.evaluate(sonucDF)
#print("Başarı : ",basari)





In [278]:
resultsetrf = ([{ 
  "Method" : methodrf,
  "Parameters" :  ("Param1=" + str(model.bestModel.getNumTrees),
                    "Param2=" + str(model.bestModel._java_obj.getMaxDepth())),
  "Accuracy" : basari                     
                  }])
deptColumns = ["Accuracy","Method","Parameters"]
resultsetrf = spark.createDataFrame(data=resultsetrf, schema = deptColumns)
resultsetrf.createOrReplaceTempView("random_forest_result")
resultsetrf = spark.sql("select Method, Parameters, Accuracy from random_forest_result").createOrReplaceTempView("random_forest_result")

In order to union the results of the 3 classification alghorithm the function is created below

In [279]:
def unioner():
 global result
 result = spark.sql("""
              select * from one_vs_rest_result
                union all
              select * from decisiontreeresults
                union all
              select * from random_forest_result
              order by Accuracy desc
              
  """).show(truncate = False)
 return result
unioner() 

+------------------------+------------------------+-------------------+
|Method                  |Parameters              |Accuracy           |
+------------------------+------------------------+-------------------+
|Random Forest Classifier|{Param1=7, Param2=8}    |0.6666666666666666 |
|Decision Tree Classifier|{Param1=0.7, Param2=0.3}|0.23157894736842105|
|One-vs-Rest classifier  |{Param1=10, Param2=7}   |0.04285714285714286|
+------------------------+------------------------+-------------------+

